# 📚 LangChain com RAG (Retrieval-Augmented Generation)

O **RAG** combina **busca de informações** com **modelos de linguagem** para gerar respostas mais precisas e atualizadas. Ele é ideal para:

- 🔍 Responder perguntas com base em documentos
- 🧠 Sistemas de QA (Pergunta e Resposta)
- 🗂️ Chatbots com base em conhecimento específico

---

### O RAG (Retrieval-Augmented Generation) é uma evolução moderna das abordagens que você já usou anteriormente com LangChain, como:

Chains (LLMChain, SequentialChain, RouterChain)
Memórias (ConversationBufferMemory, SummaryMemory, etc.)
RouterChain para direcionar fluxos
Prompt personalizado para controle de saída
🧠 O que torna o RAG mais moderno?
RAG integra busca de conhecimento externo (como documentos, bases de dados, APIs) diretamente no fluxo de geração de texto. Isso permite:

Tradicional LangChain	RAG com LangChain
Usa apenas o LLM e memória	Usa LLM + busca inteligente
Depende do contexto interno	Consulta fontes externas
Bom para conversas	Excelente para perguntas baseadas em conhecimento
Fluxo fixo	Fluxo dinâmico e adaptável

## 🧱 Componentes do RAG

1. **Documentos**: Fonte de conhecimento (PDFs, textos, bases de dados).
2. **Embeddings**: Representação vetorial dos documentos.
3. **Vector Store**: Armazena os embeddings (ex: FAISS).
4. **Retriever**: Recupera documentos relevantes.
5. **LLM**: Gera a resposta com base nos documentos recuperados.

---

## 🧪 Exemplo com Gemini

```bash
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# 1. Carrega os documentos
loader = TextLoader("meus_documentos.txt")
documents = loader.load()

# 2. Cria os embeddings com Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 3. Cria o vetor de busca (FAISS)
vectorstore = FAISS.from_documents(documents, embeddings)

# 4. Cria o retriever
retriever = vectorstore.as_retriever()

# 5. Inicializa o modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# 6. Cria a cadeia RAG (Retrieval + QA)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 7. Faz uma pergunta
query = "Quais são os principais benefícios da energia solar?"
result = rag_chain({"query": query})

# Exibe a resposta
print(result["result"])
```

### 2. 🧠 Conversational RAG (ConversationalRetrievalChain)
Mantém o contexto da conversa enquanto busca informações.

```bash
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Memória para manter o histórico
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Cadeia conversacional com busca
conversational_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

# Consulta com histórico
response = conversational_rag.run({"question": "E quais são os desafios?", "chat_history": []})


```
### 3. 🧪 Custom RAG com Prompt Personalizado
Permite usar um prompt específico para controlar como os documentos são usados.

```bash
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt customizado
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Use o contexto abaixo para responder à pergunta.
    Contexto: {context}
    Pergunta: {question}
    Resposta:"""
)

# Cadeia com prompt customizado
custom_rag = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt}
)

response = custom_rag.run("Como funciona a energia solar?")


```


### 4. 🧱 Multi-Document RAG (MapReduce ou Refine)
Divide documentos longos em partes e processa em etapas.
```bash
# MapReduce: processa partes e depois combina
map_reduce_rag = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="map_reduce"
)

# Refine: gera uma resposta inicial e refina com mais contexto
refine_rag = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="refine"
)


```

### 5. 🧭 RAG com RouterChain
Roteia perguntas para diferentes bases de conhecimento ou cadeias.

```bash
from langchain.chains.router import MultiRetrievalQAChain

# Dois retrievers diferentes
retriever_energia = retriever  # Base de energia
retriever_saude = FAISS.from_documents(outros_docs, embeddings).as_retriever()

# Mapeamento de tópicos
retrievers = {
    "energia": retriever_energia,
    "saude": retriever_saude
}

# Cadeia com roteamento
router_rag = MultiRetrievalQAChain.from_retrievers(llm=llm, retriever_names_to_retrievers=retrievers)

response = router_rag.run("Quais são os efeitos da energia solar na saúde?")


```


# 📄 LangChain - DocumentLoader

O `DocumentLoader` é responsável por **carregar documentos** de diversas fontes e formatos para serem processados por embeddings, vetores e LLMs.

---

## 🧩 Tipos Comuns de DocumentLoader

### 1. 📃 TextLoader
Carrega arquivos `.txt`.

```bash
from langchain.document_loaders import TextLoader

loader = TextLoader("documento.txt")
documents = loader.load()


In [ ]:
from langchain_community.document_loaders import TextLoader

arquivo="../docs/apostila.txt"

loader=TextLoader(arquivo)

document=loader.load()

len(document)

In [ ]:
document[0].page_content



In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai.chat_models import ChatOpenAI

Chat=ChatOpenAI()

chain=load_qa_chain(llm=Chat, chain_type="stuff", verbose=True)

In [ ]:
pergunta="o que é esse documneto"

Carregamento de Videos do Youtube

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

# Carrega a transcrição do vídeo
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/shorts/J9ss7uMYKlQ",
    add_video_info=True
)

documents = loader.load()

# Agora você pode usar os documentos com embeddings e RAG
print(documents[0].page_content)


In [ ]:
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA

# Embeddings e vetor
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# Modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Cadeia RAG
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Pergunta sobre o vídeo
response = rag_chain.run("Quais são os principais tópicos abordados no vídeo?")
print(response)


Texto Splites

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
import string

texto="Desenvolvedor com foco em IA Generativa e Data & AI, aplicando soluções inteligentes baseadas em modelos de linguagem (LLMs), NLP e arquitetura em nuvem. Atua no desenvolvimento de APIs com Python (FastAPI, Flask) e Node.js, integrando LLMs com LangChain, Amazon Bedrock, Amazon Lex e SageMaker. Trabalha com bancos relacionais (MySQL, RDS) e NoSQL (MongoDB, DynamoDB), além de pipelines de dados com Glue, Lambda e S3. Possui experiência com Docker, EC2, API Gateway e arquitetura serverless. No front-end, utiliza React e TypeScript para interfaces ricas em experiência. Participa de squads ágeis com Scrum, colaborando em soluções de ponta a ponta. Certificado AWS Cloud Practitioner e GenAI Technical. Busca consolidar-se como desenvolvedor especialista em IA e Dados, entregando valor com soluções robustas, escaláveis e baseadas em cloud."

In [ ]:
chunkSize = 50
chunkOverlap = 0

char_split = CharacterTextSplitter(
chunk_size=chunkSize,
chunk_overlap=chunkOverlap,
separator= ""
)

In [ ]:

texto="".join(f"{string.ascii_lowercase}" for _ in range(5))

len(texto)

In [ ]:
split = char_split.split_text(texto)

In [ ]:
split

In [ ]:
texto

In [ ]:
chunkSize = 50
chunkOverlap = 0

char_split = CharacterTextSplitter(
chunk_size=chunkSize,
chunk_overlap=chunkOverlap,
separator= ""
)

split = char_split.split_text(texto)

split

### Vector _ storages


In [ ]:
from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
file= "../docs/apostila.txt"
loader= TextLoader(file)
pages=loader.load()

In [ ]:
len(pages)

In [ ]:
recur_split=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["/n/n","/n", ".", " ", ""]
)

In [ ]:
document=recur_split.split_documents(pages)
for i in document:
    print(i.page_content)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma  # Corrigido de "Choroma" para "Chroma"

# Corrigido nome da variável
embedding_model = OpenAIEmbeddings()

diretorio = "../files/chroma_vectostores"  # Corrigido "choma_vectostores"

# Corrigido nome da classe e argumentos da função
vector_store = Chroma.from_documents(
    documents=document,  # Corrigido "document" para "documents"
    embedding=embedding_model,  # Corrigido "embeddings, embeddinf_model"
    persist_directory=diretorio  # Corrigido "persist_direco"
)


In [ ]:
vector_store.collection.cout()

In [ ]:

# Inicializa o vector store
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory=diretorio
)

# Consulta
pergunta = "principais manipulação de strings"  # Corrigido "princiapis manipilção"

# Busca por similaridade
docs = vector_store.similarity_search(pergunta, k=5)

# Exibe os resultados
for doc in docs:
    print(doc.page_content)
    print(f"=== {doc.metadata}\n")  # Corrigido "/n" para "\n"



## 📌 O que é um Retriever?
Um retriever é uma interface que, dado um texto de entrada (como uma pergunta), retorna os documentos mais relevantes de um repositório (como um VectorStore). Ele é usado principalmente em sistemas de RAG (Retrieval-Augmented Generation), onde a IA busca informações antes de gerar uma resposta.

✅ Exemplo básico com Chroma e LangChain:


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Criação do modelo de embeddings
embedding_model = OpenAIEmbeddings()

# Carregamento do vector store
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory="../files/chroma_vectostores"
)

# Criação do retriever a partir do vector store
retriever = vector_store.as_retriever()

# Consulta
query = "principais funções de manipulação de strings"
docs = retriever.get_relevant_documents(query)

# Exibição dos resultados
for doc in docs:
    print(doc.page_content)
    print(f"=== {doc.metadata}\n")


A busca semântica é uma técnica de recuperação de informações que entende o significado das palavras em vez de apenas procurar por correspondência exata de termos. Isso é especialmente útil quando queremos encontrar documentos ou respostas que são conceitualmente relevantes, mesmo que não usem exatamente as mesmas palavras da consulta.

🧠 Diferença entre busca tradicional e busca semântica:
Tipo de Busca	Como funciona	Exemplo
Tradicional	Procura palavras exatas no texto	Consulta: "manipulação de strings" só retorna documentos com essas palavras
Semântica	Usa embeddings para entender o significado	Consulta: "como trabalhar com texto" pode retornar documentos sobre manipulação de strings
🔧 Como funciona na prática com LangChain:
Documentos são convertidos em vetores usando um modelo de embeddings (como OpenAIEmbeddings).
A consulta do usuário também é convertida em vetor.
O sistema compara os vetores usando similaridade de cosseno ou outra métrica.
Retorna os documentos mais semânticamente próximos.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Modelo de embeddings
embedding_model = OpenAIEmbeddings()

# Carrega o vetor store
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory="../files/chroma_vectostores"
)

# Busca semântica
query = "como trabalhar com texto em Python"
docs = vector_store.similarity_search(query, k=3)

# Exibe os resultados
for doc in docs:
    print(doc.page_content)
    print(f"=== {doc.metadata}\n")


🔍 max_relevance — O que é?
Em alguns retrievers (como os do LangChain), você pode configurar o modo de busca para priorizar relevância máxima. Isso significa que o sistema será mais rigoroso ao escolher os documentos mais próximos semanticamente da sua consulta.

✅ Como usar no LangChain
Se estiver usando um retriever criado com .as_retriever(), você pode configurar o modo assim:


In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "lambda_mult": 0.8}
)


A filtragem na busca semântica é o processo de limitar ou refinar os resultados com base em critérios específicos, como metadados, tags, datas, categorias, etc. Isso é útil quando você quer buscar documentos relevantes dentro de um contexto específico.

🔍 Exemplo de filtragem com Chroma e LangChain
Suponha que seus documentos tenham metadados como "categoria": "Python" ou "autor": "Cicero".

Você pode aplicar filtros assim:



In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {"categoria": "Python"}  # Filtra por metadado
    }
)

query = "como manipular strings"
docs = retriever.get_relevant_documents(query)

for doc in docs:
    print(doc.page_content)
    print(f"=== {doc.metadata}\n")
